# Create database on primary

Set recovery to full and backup to allow add to AG

Add to AG manually

In [5]:
USE master;

DROP DATABASE IF EXISTS TestDb;

CREATE DATABASE TestDb;

ALTER DATABASE TestDb SET RECOVERY FULL;

BACKUP DATABASE TestDb TO DISK = N'NUL';


Processed 376 pages for database 'TestDb', file 'TestDb' on file 1.

Processed 2 pages for database 'TestDb', file 'TestDb_log' on file 1.

BACKUP DATABASE successfully processed 378 pages in 0.120 seconds (24.576 MB/sec).

Total execution time: 00:00:01.974

# Create sequence and table
Set SeqId default to next value for sequence

In [6]:
USE TestDb;

CREATE SEQUENCE dbo.CountBy1  
    START WITH 1  
    INCREMENT BY 1 ;  
GO

CREATE TABLE dbo.SequenceTest
(SeqId INT
,TextCol NVARCHAR(100))

ALTER TABLE [dbo].[SequenceTest] ADD  CONSTRAINT [DF_SeqID]  DEFAULT (NEXT VALUE FOR [CountBy1]) FOR [SeqID]
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.015

# Insert row to table

In [7]:
USE TestDb;

INSERT INTO dbo.SequenceTest
(TextCol)
VALUES
(N'test')

(1 row affected)

Total execution time: 00:00:00.007

# Check sys.sequences view on primary
Add database to the AG

In [8]:
USE TestDb;

SELECT name, current_Value, last_used_value FROM sys.sequences 
WHERE name =  N'CountBy1' ORDER BY name

(1 row affected)

Total execution time: 00:00:00.014

name,current_Value,last_used_value
CountBy1,1,1


# Check sys.sequences view on secondary using ApplicationIntent=ReadOnly


In [9]:
USE TestDb;

SELECT name, current_Value, last_used_value FROM sys.sequences 
WHERE name =  N'CountBy1' ORDER BY name

(1 row affected)

Total execution time: 00:00:00.015

name,current_Value,last_used_value
CountBy1,50,50


# Insert second row to table

In [2]:
USE TestDb;

INSERT INTO dbo.SequenceTest
(TextCol)
VALUES
(N'test')

(1 row affected)

Total execution time: 00:00:00.004

# Check sys.sequences view on primary

In [3]:
USE TestDb;

SELECT name, current_Value, last_used_value FROM sys.sequences 
WHERE name =  N'CountBy1' ORDER BY name

(1 row affected)

Total execution time: 00:00:00.012

name,current_Value,last_used_value
CountBy1,2,2


# Check sequences view on secondary with ApplicationIntent=ReadOnly

In [10]:
USE TestDb;

SELECT name, current_Value, last_used_value FROM sys.sequences 
WHERE name =  N'CountBy1' ORDER BY name

(1 row affected)

Total execution time: 00:00:00.012

name,current_Value,last_used_value
CountBy1,50,50


# Manually fail the AG over

# Check sys.sequences on the new primary

In [1]:
USE TestDb;

SELECT name, current_Value, last_used_value FROM sys.sequences 
WHERE name =  N'CountBy1' ORDER BY name

(1 row affected)

Total execution time: 00:00:00.008

name,current_Value,last_used_value
CountBy1,50,50


# Check sys.sequences on the new secondary with ApplicationIntent=ReadOnly

In [2]:
USE TestDb;

SELECT name, current_Value, last_used_value FROM sys.sequences 
WHERE name =  N'CountBy1' ORDER BY name

(1 row affected)

Total execution time: 00:00:00.013

name,current_Value,last_used_value
CountBy1,50,50
